In [58]:
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Data augmentation
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_data_gen = ImageDataGenerator(rescale=1./255)

# Load pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Fine-tuning last few layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Add custom top layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

# Compile Model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Learning Rate Reduction and Early Stopping Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.000001)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Load training and validation data
train_generator = train_data_gen.flow_from_directory(
    'data -Bee/train',
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical'
)

validation_generator = validation_data_gen.flow_from_directory(
    'data -Bee/test',
    target_size=(48, 48),
    batch_size=64,
    class_mode='categorical'
)

# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[reduce_lr, early_stopping]
)

# Save Model Architecture to JSON
model_json = model.to_json()
with open("Activity_transfer_learning.json", "w") as json_file:
    json_file.write(model_json)

# Save Trained Model Weights
model.save_weights('Activity_transfer_learning.h5')

# Evaluation on Test Dataset
test_loss, test_accuracy = model.evaluate_generator(validation_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 251 images belonging to 5 classes.
Found 251 images belonging to 5 classes.
Epoch 1/50


C:\Users\satvi\AppData\Local\Temp\ipykernel_4344\2853267584.py:65: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


3/3 [==============================] - 4s 1s/step - loss: 1.9243 - accuracy: 0.2086 - val_loss: 1.5870 - val_accuracy: 0.1823 - lr: 1.0000e-04
Epoch 2/50
3/3 [==============================] - 3s 1s/step - loss: 1.8254 - accuracy: 0.1979 - val_loss: 1.5660 - val_accuracy: 0.2865 - lr: 1.0000e-04
Epoch 3/50
3/3 [==============================] - 3s 974ms/step - loss: 1.7500 - accuracy: 0.2032 - val_loss: 1.5544 - val_accuracy: 0.3698 - lr: 1.0000e-04
Epoch 4/50
3/3 [==============================] - 3s 946ms/step - loss: 1.5962 - accuracy: 0.2834 - val_loss: 1.5591 - val_accuracy: 0.3958 - lr: 1.0000e-04
Epoch 5/50
3/3 [==============================] - 3s 979ms/step - loss: 1.6491 - accuracy: 0.2031 - val_loss: 1.5484 - val_accuracy: 0.4375 - lr: 1.0000e-04
Epoch 6/50
3/3 [==============================] - 3s 966ms/step - loss: 1.6443 - accuracy: 0.2246 - val_loss: 1.5354 - val_accuracy: 0.4271 - lr: 1.0000e-04
Epoch 7/50
3/3 [==============================] - 3s 989ms/step - loss: 1.5

C:\Users\satvi\AppData\Local\Temp\ipykernel_4344\2853267584.py:83: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy = model.evaluate_generator(validation_generator)


Test Loss: 0.6065680980682373
Test Accuracy: 0.8167330622673035
